In [1]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch.optim as optim
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import cv2
import os
from keras.preprocessing import image
from keras.applications.resnet import ResNet50, preprocess_input
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding, LSTM, Dense, Dropout, GlobalAveragePooling2D, GlobalMaxPooling2D, concatenate, Input, Flatten, BatchNormalization
import torch.utils.data as data
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torch
import torchvision.models as models
from torch.nn.utils.rnn import pad_sequence
import gensim.downloader as api


In [2]:
df = pd.read_csv('sentiment.csv')
df.drop(df.columns[0], axis=1, inplace=True)
df.head()

,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.


# Text Preprocessing

In [3]:
# Define a lambda function to remove quotes from words in a list
def remove_quotes(lst): return [word.strip("'") for word in lst]

# Apply the lambda function to the 'text' column of the dataframe
df['tokens'] = df['tokens'].apply(lambda x: remove_quotes(eval(x)))

# Print the resulting dataframe
df


,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"[a, plate, of, delicious, food, including, fre...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"[french, fries, are, not, a, healthy, food, bu...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"[the, plate, has, one, of, my, favorite, foods...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"[it, was, disgusting, food, not, just, bad, food]","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"[a, plate, of, disgusting, food, found, at, a,...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...
39194,24628,test,COCO_val2014_000000190705.jpg,1,"[a, dirty, bathroom, that, has, a, dirty, wind...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,24628,test,COCO_val2014_000000190705.jpg,1,"[a, dirty, bathroom, that, has, a, window, in,...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,24628,test,COCO_val2014_000000190705.jpg,1,"[a, towel, that, is, on, a, rack, in, a, dirty...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,24628,test,COCO_val2014_000000190705.jpg,1,"[a, dirty, bathroom, that, has, a, dirty, wind...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


In [4]:
# Load the GloVe word embeddings
# Load the 'glove-wiki-gigaword-100' embeddings
glove_model = api.load('glove-wiki-gigaword-100')


def get_word_embeddings(word):
    if word in glove_model:
        return glove_model[word]
    else:
        return np.zeros(100)


def embedding(tokens):
    embeddings = np.array([get_word_embeddings(word) for word in tokens])
    return embeddings


# Convert sample_captions['cleaned_caption'] to numerical_caption using embeddings
df['text_features'] = df['tokens'].apply(embedding)

df['text_features'] = df['text_features'].apply(torch.tensor)

df.head(2)


,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw,text_features
0,31369,train,COCO_val2014_000000389081.jpg,1,"[a, plate, of, delicious, food, including, fre...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.,"[[tensor(-0.2709), tensor(0.0440), tensor(-0.0..."
1,31369,train,COCO_val2014_000000389081.jpg,1,"[french, fries, are, not, a, healthy, food, bu...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...,"[[tensor(0.0270), tensor(-0.0538), tensor(0.16..."


# Resizing and normalizing the images

In [5]:
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])
model.eval()
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


c:\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
def extract_features(folder_path):
    features = {}
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img)
        img_tensor = img_tensor.unsqueeze(0)
        with torch.no_grad():
            features_tensor = model(img_tensor)
        features_np = torch.squeeze(features_tensor)
        features[file] = features_np
    return features


In [7]:
folder_path = 'sentiment_images'
image_features = extract_features(folder_path)


In [8]:


# Create a new list to store the values from the dictionary
imageFeatures = []

# Iterate through the rows of the DataFrame
for index, row in df.iterrows():
    # Check if the value in 'col1' matches a key in the dictionary
    if row['filename'] in image_features:
        # If there is a match, append the corresponding value to the new list
        imageFeatures.append(image_features[row['filename']])
    else:
        # If there is no match, append NaN to the new list
        imageFeatures.append(torch.tensor(float('nan')))
        


# Add the new list as a new column in the DataFrame
df['image_features'] = imageFeatures


In [9]:
df_copy = df.copy()


In [12]:
df_copy = df_copy[df_copy.apply(lambda x: torch.tensor(x['image_features']).numel() != 0, axis=1)]


C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\1327902967.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  df_copy = df_copy[df_copy.apply(lambda x: torch.tensor(x['image_features']).numel() != 0, axis=1)]


In [13]:
# Drop rows with empty tensors
for i, features in enumerate(df_copy['image_features']):
    if torch.tensor(features).shape == torch.Size([]):
        df_copy.drop(i, inplace=True)


C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\3422520640.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if torch.tensor(features).shape == torch.Size([]):
C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\3422520640.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if torch.tensor(features).shape == torch.Size([]):
C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\3422520640.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if torch.tensor(features).shape == torch.Size([]):
C:\Users\Aeiman Imtiaz\AppDa

# Test Train Split

In [122]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df_copy, test_size=0.2, random_state=1)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Print the shapes of the training and testing data
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)


Training data shape: (31287, 10)
Testing data shape: (7822, 10)


# DataLoaders

In [123]:
import math

# Define a function to create the dataset


def create_dataset(data, batch_size):
    image_features = torch.stack(
        [torch.tensor(features) for features in data['image_features'].values])
    captions = [caption[:100] if len(
        caption) > 100 else caption for caption in data['text_features'].values]
    text_features = pad_sequence([torch.tensor(
        caption) for caption in captions], batch_first=True, padding_value=0)
    labels = torch.tensor(data['sentiment'])

    # Define a function to get a batch from the dataset
    def get_batch(index):
        start_index = index * batch_size
        end_index = (index + 1) * batch_size
        text_batch = text_features[start_index:end_index]
        image_batch = image_features[start_index:end_index]
        label_batch = labels[start_index:end_index]

        return text_batch, image_batch, label_batch

    num_samples = len(labels)
    num_batches = int(math.ceil(num_samples / batch_size))

    return get_batch, num_batches


batch_size = 16

# Create train and validation datasets
train_get_batch, num_train_batches = create_dataset(train_data, batch_size)
test_get_batch, num_test_batches = create_dataset(test_data, batch_size)

# Define the dataloaders using the dataset functions
train_dataloader = DataLoader(
    range(num_train_batches), batch_size=1, shuffle=True,
    collate_fn=lambda x: train_get_batch(x[0]))
test_dataloader = DataLoader(
    range(num_test_batches), batch_size=1,
    collate_fn=lambda x: test_get_batch(x[0]))


C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\2505105041.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(features) for features in data['image_features'].values])
C:\Users\Aeiman Imtiaz\AppData\Local\Temp\ipykernel_14472\2505105041.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  text_features = pad_sequence([torch.tensor(


# Model for Sentiment Analysis

In [124]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model architecture


class MultiModalSentimentAnalysis(nn.Module):
    def __init__(self, text_input_size, image_input_size, hidden_size, num_classes):
        super(MultiModalSentimentAnalysis, self).__init__()

        # Text processing module
        self.text_lstm = nn.LSTM(
            text_input_size, hidden_size, batch_first=True)

        # Image processing module
        self.image_cnn = nn.Sequential(
            nn.Linear(image_input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, hidden_size)
        )

        # Fully-connected layer for prediction
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, text_features, image_features):
        # Process text features
        text_output, _ = self.text_lstm(text_features)
        text_output = text_output[:, -1, :]  # take last output of LSTM

        # Process image features
        image_output = self.image_cnn(image_features)

        # Concatenate text and image outputs
        combined_output = torch.cat((text_output, image_output), dim=1)

        # Make prediction
        logits = self.fc(combined_output)
        return logits


# Instantiate the model
model = MultiModalSentimentAnalysis(
    text_input_size=100, image_input_size=2048, hidden_size=256, num_classes=2)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
num_epochs = 10
batch_size = 16
# Train the model
for epoch in range(num_epochs):
    epoch_loss = 0.0
    epoch_acc = 0.0
    num_batches = 0

    # Loop over the batches in the dataset
    for text_batch, image_batch, label_batch in train_dataloader:
        # Forward pass
        outputs = model(text_batch, image_batch)
        loss = criterion(outputs, label_batch)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct = (predicted == label_batch).sum().item()
        total = label_batch.size(0)
        accuracy = correct / total

        # Accumulate loss and accuracy for the epoch
        epoch_loss += loss.item()
        epoch_acc += accuracy
        num_batches += 1

    # Print metrics for the epoch
    epoch_loss /= num_batches
    epoch_acc /= num_batches
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch +
          1, num_epochs, epoch_loss, epoch_acc*100))


Epoch [1/10], Loss: 0.6908, Accuracy: 54.25%
Epoch [2/10], Loss: 0.6892, Accuracy: 54.45%
Epoch [3/10], Loss: 0.6874, Accuracy: 54.40%
Epoch [4/10], Loss: 0.6850, Accuracy: 54.42%
Epoch [5/10], Loss: 0.6823, Accuracy: 54.42%
Epoch [6/10], Loss: 0.6803, Accuracy: 54.31%
Epoch [7/10], Loss: 0.6794, Accuracy: 53.77%
Epoch [8/10], Loss: 0.6765, Accuracy: 53.44%
Epoch [9/10], Loss: 0.6715, Accuracy: 53.59%
Epoch [10/10], Loss: 0.6694, Accuracy: 53.86%


In [126]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Evaluate the model on the test set
model.eval()
y_true = []
y_pred = []

for text_batch, image_batch, label_batch in test_dataloader:
    # Make predictions
    outputs = model(text_batch, image_batch)
    _, predicted = torch.max(outputs, 1)

    # Store the ground-truth labels and predicted labels
    y_true += label_batch.tolist()
    y_pred += predicted.tolist()

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('Precision: {:.2f}%'.format(precision * 100))
print('Recall: {:.2f}%'.format(recall * 100))
print('F1-score: {:.2f}%'.format(f1 * 100))


Accuracy: 53.58%
Precision: 53.58%
Recall: 100.00%
F1-score: 69.77%


# Task 3

In [127]:
positive = df_copy['sentiment'] > 0
df_positive = df_copy[positive]


In [128]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(
    df_positive, test_size=0.2, random_state=1)
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

# Print the shapes of the training and testing data
print("Training data shape:", train_data.shape)
print("Testing data shape:", test_data.shape)


Training data shape: (16983, 10)
Testing data shape: (4246, 10)


In [131]:

# Define the model architecture
class MultiModalCaptionGeneration(nn.Module):
    def __init__(self, text_input_size, image_input_size, hidden_size, vocab_size):
        super(MultiModalCaptionGeneration, self).__init__()

        # Text processing module
        self.text_lstm = nn.LSTM(
            text_input_size, hidden_size, batch_first=True)

        # Image processing module
        self.image_cnn = nn.Sequential(
            nn.Linear(image_input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, hidden_size)
        )

        # Fully-connected layer for prediction
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, text_features, image_features, caption=None):
        # Process text features
        text_output, _ = self.text_lstm(text_features)
        text_output = text_output[:, -1, :]  # take last output of LSTM

        # Process image features
        image_output = self.image_cnn(image_features)

        # Concatenate text and image outputs
        combined_output = torch.cat((text_output, image_output), dim=1)

        if caption is not None:
            # Training mode - predict next word in caption
            outputs = self.fc(combined_output)
            loss = nn.CrossEntropyLoss()(outputs, caption)
            return loss
        else:
            # Testing mode - generate caption
            caption = []
            for _ in range(20):  # maximum length of caption
                outputs = self.fc(combined_output)
                _, predicted = torch.max(outputs, 1)
                caption.append(predicted.item())
                if predicted.item() == 2:  # end of sentence token
                    break
                word_embedding = self.fc.weight[predicted.item()].unsqueeze(0)
                text_output, _ = self.text_lstm(
                    word_embedding, text_output.unsqueeze(0))
                text_output = text_output[:, -1, :]
                combined_output = torch.cat((text_output, image_output), dim=1)
            return caption


In [133]:



# Define the dataset for training
class CaptionDataset(Dataset):
    def __init__(self, text_features, image_features, captions):
        self.text_features = text_features
        self.image_features = image_features
        self.captions = captions

    def __len__(self):
        return len(self.captions)

    def __getitem__(self, index):
        text_features = torch.tensor(
            self.text_features[index], dtype=torch.float32)
        image_features = torch.tensor(
            self.image_features[index], dtype=torch.float32)
        caption = self.captions[index]
        return text_features, image_features, caption


# Define the dataset for testing
class CaptionTestDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = 'sentiment/' + image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        return image, image_path
    
    

# Define the transforms that should be applied to each image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                            (0.229, 0.224, 0.225))
])

# Create the training and testing datasets
train_dataset = CaptionDataset(
    train_data['text_features'], train_data['image_features'], train_data['sentiment'])
test_dataset = CaptionTestDataset(
    test_data['filename'], transform=transform)

# create dataloaders
train_dataloader = DataLoader(
    train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_dataloader = DataLoader(
    test_dataset, batch_size=32, shuffle=False, num_workers=4)






In [134]:
# Define hyperparameters
text_input_size = 100  # size of GloVe embeddings
image_input_size = 2048  # size of ResNet-101 features
hidden_size = 512
vocab_size = 10000

# Instantiate model
model = MultiModalCaptionGeneration(
    text_input_size, image_input_size, hidden_size, vocab_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    epoch_loss = 0
    for i, (image_features, text_features, caption) in enumerate(train_dataloader):
        optimizer.zero_grad()
        loss = model(text_features, image_features, caption)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {epoch_loss/i:.4f}")


In [ ]:
# Set the model to evaluation mode
model.eval()

# Create lists to store the predicted captions and image paths
predicted_captions = []
image_paths = []

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Iterate over the test dataloader
for images, paths in test_dataloader:
    # Move the images to the device
    images = images.to(device)

    # Process the images using the image processing module of the model
    image_features = model.image_cnn(images)

    # Initialize the text features with zeros
    text_features = torch.zeros(
        images.shape[0], 1, model.text_lstm.hidden_size).to(device)

    # Generate captions using the model
    captions = model(text_features, image_features)

    # Convert the predicted caption indices to words
    captions = [glove_model.idx2word[idx] for idx in captions]

    # Add the predicted captions and image paths to the lists
    predicted_captions.extend(captions)
    image_paths.extend(paths)

# Write the predicted captions to a file
with open('predicted_captions.txt', 'w') as f:
    for path, caption in zip(image_paths, predicted_captions):
        f.write('{}\t{}\n'.format(path, caption))
